In [ ]:
import os
import pickle
import pandas as pd

from bert_score import score as bert_score_fn
import pickle
import evaluate
import string

In [ ]:
not_present_dic={"age": "no.","history":"no history was found.","parenchymal_distortion":"no.","nodules_echo_size":"unknown."}

In [ ]:
model="biogptQA2e55e66no_copydescription8batchfinal"
questions=["age","history","parenchymal_distortion","nodules_echo_size"]
# questions=["age","tipo","tecnica","family","history","symtomatic",
#            "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
#           "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]

In [ ]:
def truncate_before_keyword(text, keyword):
    """Truncate text before keyword (e.g., 'comparative study') and end with a period."""
    keyword_pos = text.lower().find(keyword.lower())
    if keyword_pos == -1:
        cleaned = text.strip()
    else:
        # Text before keyword
        partial = text[:keyword_pos]
        # Find last punctuation before keyword
        match = re.search(r'(.+?)[\.\,\;\n][^.,;\n]*$', partial)
        cleaned = match.group(1).strip() if match else partial.strip()
    
    # Ensure the result ends with a single period
    if not cleaned.endswith('.'):
        cleaned += '.'
    
    return cleaned

In [ ]:
def calculate_acc(pred,truth,tipo,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]   
            
            if predicted.startswith(not_present_dic[tipo]):
                predicted="no response."
            elif tipo=="nodules_echo_size":
                predicted = predicted.split("mm")[0] + "mm."
            else:
                predicted = predicted.split(".")[0] + "."
            predicted=truncate_before_keyword(predicted, "comparative study")
            if predicted.lower()==str(real).lower():
                acc+=1
            # else:
            #     print(key,"REAL: ", real,"PREDICTED: ",predicted)
        else:
            print(key)
    print(acc)
    print(total)

    return acc/total

In [ ]:
import re
import string
import evaluate

rouge = evaluate.load("rouge")
exact_match = evaluate.load("exact_match")

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = set(prediction_tokens) & set(ground_truth_tokens)
    num_same = len(common)
    if len(prediction_tokens) == 0 or len(ground_truth_tokens) == 0:
        return int(prediction_tokens == ground_truth_tokens)
    if num_same == 0:
        return 0
    precision = num_same / len(prediction_tokens)
    recall = num_same / len(ground_truth_tokens)
    return 2 * precision * recall / (precision + recall)

def calculate_metrics(pred, truth, tipo):
    preds, refs = [], []
    for key, real in truth.items():
        if key not in pred:
            continue
        predicted = pred[key]
        if tipo in not_present_dic:
            if predicted.startswith(not_present_dic[tipo]):
                predicted="no response."
            elif tipo=="nodules_echo_size":
                predicted = predicted.split("mm")[0] + "mm."
            else:
                predicted = predicted.split(".")[0] + "."
        
        predicted=truncate_before_keyword(predicted, "comparative study")
        preds.append(predicted)
        refs.append(real)
    # Compute metrics
    f1s, ems = [], []
    for p, r in zip(preds, refs):
        f1s.append(f1_score(p, r))
        ems.append(int(normalize_answer(p) == normalize_answer(r)))


    rouge_scores = rouge.compute(predictions=preds, references=refs)
    rouge_l = rouge_scores["rougeL"]

    P, R, F1 = bert_score_fn(preds, refs, lang="en", verbose=False, rescale_with_baseline=True)
    bert_f1 = F1.mean().item()
    bert_R = R.mean().item()

    print(f"Examples: {len(preds)}")
    print(f"Token F1: {sum(f1s)/len(f1s):.4f}")
    print(f"Exact Match: {sum(ems)/len(ems):.4f}")
    print(f"ROUGE-L: {rouge_l:.4f}")
    print(f"BERTScore F1: {bert_f1:.4f}")
    print(f"BERTScore R: {bert_R:.4f}")

    return {
        "token_f1": sum(f1s)/len(f1s),
        "exact_match": sum(ems)/len(ems),
        "rouge_l": rouge_l,
        "bert_score_f1": bert_f1,
        "bert_score_R": bert_R,
        "list_bert": F1,
    }

In [ ]:
model="clinicalt57e57no_copydescription16batchfinaltokenized"
f1_total=[]
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    truth = {
    k: ("no response." if v in list(not_present_dic.values()) else v)
    for k, v in truth.items()
    }
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point="mm"))
    else:
        print(calculate_acc(output,truth,tipo))

    result=calculate_metrics(output, truth, tipo)
    f1_bio=result["list_bert"]
    f1_total+=f1_bio

In [ ]:
model_name="biogpt"+"1e57"+"no_copy"+"description"+"16batch"+"final"
f1_total=[]
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    truth = {
    k: ("no response." if v in list(not_present_dic.values()) else v)
    for k, v in truth.items()
    }
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point="mm"))
    else:
        print(calculate_acc(output,truth,tipo))

    result=calculate_metrics(output, truth, tipo)
    f1_bio=result["list_bert"]
    f1_total+=f1_bio

In [ ]:
model_name="biogpt"+"21e51e57"+"no_copy"+"description"+"16batch"+"final"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model_name}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

In [ ]:
model="biogpt"+"5e65e63"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo=="history" or tipo=="family" or tipo=="symtomatic" or tipo=="density_echo" or tipo=="nodules_echo_echogenicity" or tipo=="nodules_echo_margin" or tipo=="nodules_echo_shape"  or tipo=="nodules_echo_location":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

In [ ]:
model="biogpt1e55e63"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo=="history" or tipo=="family" or tipo=="symtomatic" or tipo=="density_echo" or tipo=="nodules_echo_echogenicity" or tipo=="nodules_echo_margin" or tipo=="nodules_echo_shape"  or tipo=="nodules_echo_location":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

In [ ]:
calculate_acc(biogpt_age,truth_age)

In [ ]:
calculate_acc(biogpt_tipo,truth_tipo)

In [ ]:
calculate_acc(biogpt_tecnica,truth_tecnica)